# Test Switches SE(2)

In [ ]:
import taichi as ti
ti.init(arch=ti.gpu, debug=False, device_memory_GB=3.5) # Use less than 4 so that we don't mix RAM and VRAM (?)
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import h5py
import dsfilter

In [ ]:
save_files = True

## Lines

In [ ]:
test_case = 3

In [ ]:
match test_case:
    case 1: # Grid of lines (black)
        dim_I, dim_J = 128, 128
        u_ground_truth = np.ones((dim_I, dim_J)) * 255.
        N_lines = 4
        offset = dim_I // (N_lines + 1)
        for k in range(N_lines):
            centre = (k + 1) * offset
            u_ground_truth[(centre-1):(centre+2), :] = 0.
            u_ground_truth[:, (centre-1):(centre+2)] = 0.

        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        mask = (xs**2 + ys**2) < 0.6
        u = u_ground_truth.copy()
        u[mask] = 0.5 * 255.

    case 2: # Grid of lines (alternating black and white)
        dim_I, dim_J = 128, 128
        u_ground_truth = np.ones((dim_I, dim_J)) * 0.5 * 255.
        N_lines = 7
        offset = dim_I // (N_lines + 1)
        colour = 0. # black
        for k in range(N_lines):
            centre = (k + 1) * offset
            u_ground_truth[(centre-1):(centre+2), :] = colour
            u_ground_truth[:, (centre-1):(centre+2)] = 255. - colour
            colour = 255. - colour
        u_ground_truth = sp.ndimage.gaussian_filter(u_ground_truth, 1)

        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        mask = (xs**2 + ys**2) < 0.5
        u = u_ground_truth.copy()
        u[mask] = 0.5 * 255.

    case 3: # Horizontal lines (black)
        dim_I, dim_J = 128, 128
        u_ground_truth = np.ones((dim_I, dim_J)) * 255.
        N_lines = 4
        offset = dim_I // (N_lines + 1)
        for k in range(N_lines):
            centre = (k + 1) * offset
            u_ground_truth[:, (centre-1):(centre+2)] = 0.

        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        mask = (xs**2 + ys**2) < 0.6
        u = u_ground_truth.copy()
        u[mask] = 1. * 255.


u = sp.ndimage.gaussian_filter(u, 1.) # Smooth for well-posed lifting.

mask = 1 - mask.astype(int)
mask = sp.ndimage.binary_erosion(mask, iterations=8, border_value=1).astype(int) # Deal with boundary artefacts.

dim_I, dim_J = u.shape
dim_K = 16
Is, Js, Ks = np.indices((dim_I, dim_J, dim_K))
x_min, x_max = 0., dim_I - 1.
y_min, y_max = 0., dim_J - 1.
θ_min, θ_max = 0., 2 * np.pi
dxy = (x_max - x_min) / (dim_I - 1)
dθ = (θ_max - θ_min) / dim_K
xs, ys, θs = dsfilter.SE2.utils.coordinate_array_to_real(Is, Js, Ks, x_min, y_min, θ_min, dxy, dθ)

In [ ]:
fig, ax, cbar = dsfilter.visualisations.plot_image_array(u, x_min, x_max, y_min, y_max, cmap="gray")
ax.set_title("$u$")
fig.colorbar(cbar, ax=ax);

## Check Orientation Score

In [ ]:
cws_check = dsfilter.orientationscore.cakewavelet_stack(dim_I, dim_K, Gaussian_σ=dim_I/8)

In [ ]:
K = 1
print(θs[0, 0, K])
fig, ax, cbar = dsfilter.visualisations.plot_image_array(cws_check.real[K], x_min, x_max, y_min, y_max, cmap="gray")
ax.set_title("$\psi$")
fig.colorbar(cbar, ax=ax);

In [ ]:
cws = dsfilter.orientationscore.cakewavelet_stack(dim_I, dim_K, Gaussian_σ=dim_I/8)
U = dsfilter.orientationscore.wavelet_transform(u, cws.real)
U = np.transpose(U, axes=(1, 2, 0)) # x, y, θ
Mask = np.transpose(np.array([mask for _ in range(dim_K)]), axes=(1, 2, 0)) # x, y, θ
U.shape, Mask.shape

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(1, 3, figsize=(18, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title(f"{K} $U$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title(f"{K + 2} $U$")
fig.colorbar(cbar, ax=ax[1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2])
ax[2].set_title(f"{K + 4} $U$")
fig.colorbar(cbar, ax=ax[2]);

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(u, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title("$u$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title("Reconstruction")
fig.colorbar(cbar, ax=ax[1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u - U.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2])
ax[2].set_title("Reconstruction error")
fig.colorbar(cbar, ax=ax[2]);

In [ ]:
if save_files:
    filename = f".\\data\\test_switches_SE2_os.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=U)

## Presmooth

In [ ]:
from dsfilter.R2.filter import diffusion_inpainting as diffusion_inpainting_R2
from dsfilter.SE2.LI.filter import diffusion_inpainting as diffusion_inpainting_SE2

In [ ]:
u_pre_R2 = diffusion_inpainting_R2(u, mask, dxy, 2000)
U_pre_R2 = dsfilter.orientationscore.wavelet_transform(u_pre_R2, cws.real)
U_pre_R2 = np.transpose(U_pre_R2, axes=(1, 2, 0)) # x, y, θ
U_pre_SE2 = diffusion_inpainting_SE2(U, Mask, np.array((1., 0., 0.)), dxy, dθ, θs, 2000)
U_pre_delete = np.zeros_like(U)
for k in range(dim_K):
    U_pre_delete[..., k] += mask * U[..., k] + (1 - mask) * U[..., k].mean()

In [ ]:
K = 4
print(θs[0, 0, K])
fig, ax = plt.subplots(1, 4, figsize=(24, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title(f"{K} $U$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_pre_R2[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title(f"{K} $U$ Presmoothed $\mathbb{{R}}^2$")
fig.colorbar(cbar, ax=ax[1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_pre_SE2[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2])
ax[2].set_title(f"{K} $U$ Presmoothed $\mathrm{{SE}}(2)$")
fig.colorbar(cbar, ax=ax[2])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_pre_delete[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[3])
ax[3].set_title(f"{K} $U$ Deleted Mask")
fig.colorbar(cbar, ax=ax[3]);

In [ ]:
test_R2 = U_pre_R2.copy()
test_R2[Mask > 0.] = 15.
test_SE2 = U_pre_SE2.copy()
test_SE2[Mask > 0.] = 15.
test_delete = U_pre_delete.copy()
test_delete[Mask > 0.] = 15.

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(3, 3, figsize=(18, 15))
_, _, cbar = dsfilter.visualisations.plot_image_array(test_R2[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"{K} $U$ Presmoothed $\mathbb{{R}}^2$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_R2[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"{K + 2} $U$ Presmoothed $\mathbb{{R}}^2$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_R2[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title(f"{K + 4} $U$ Presmoothed $\mathbb{{R}}^2$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_SE2[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"{K} $U$ Presmoothed $\mathrm{{SE}}(2)$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_SE2[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"{K + 2} $U$ Presmoothed $\mathrm{{SE}}(2)$")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_SE2[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title(f"{K + 4} $U$ Presmoothed $\mathrm{{SE}}(2)$")
fig.colorbar(cbar, ax=ax[1, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_delete[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 0])
ax[2, 0].set_title(f"{K} $U$ Deleted Mask")
fig.colorbar(cbar, ax=ax[2, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_delete[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 1])
ax[2, 1].set_title(f"{K + 2} $U$ Deleted Mask")
fig.colorbar(cbar, ax=ax[2, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_delete[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2, 2])
ax[2, 2].set_title(f"{K + 4} $U$ Deleted Mask")
fig.colorbar(cbar, ax=ax[2, 2]);

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(U_pre_R2.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("Reconstruction Presmoothed $\mathbb{{R}}^2$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(u - U_pre_R2.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("Reconstruction error Presmoothed $\mathbb{{R}}^2$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_pre_SE2.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("Reconstruction Presmoothed $\mathrm{{SE}}(2)$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u - U_pre_SE2.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title("Reconstruction error Presmoothed $\mathrm{{SE}}(2)$")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_pre_delete.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title("Reconstruction Deleted Mask")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(u - U_pre_delete.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title("Reconstruction error Deleted Mask")
fig.colorbar(cbar, ax=ax[1, 2]);

## Check Internal Regularisation

In [ ]:
from dsfilter.SE2.regularisers import (
    gaussian_derivative_kernel,
    convolve_with_kernel_x_dir,
    convolve_with_kernel_y_dir,
    convolve_with_kernel_θ_dir
)
from dsfilter.SE2.regularisers import regularise_anisotropic as regularise_anisotropic_int

In [ ]:
def regularise_isotropic(U_np, ν_s, ν_o, dxy, dθ):
    shape = U_np.shape
    U = ti.field(dtype=ti.f32, shape=shape)
    U.from_numpy(U_np)
    k_s, radius_s = gaussian_derivative_kernel(ν_s, 0, dxy=dxy)
    k_o, radius_o = gaussian_derivative_kernel(ν_o, 0, dxy=dθ)
    convolution_storage = ti.field(dtype=ti.f32, shape=shape)
    U_reg = ti.field(dtype=ti.f32, shape=shape)
    regularise_isotropic_ti(U, k_s, radius_s, k_o, radius_o, convolution_storage, U_reg)
    return U_reg.to_numpy()

@ti.kernel
def regularise_isotropic_ti(
    U: ti.template(),
    k_s: ti.template(),
    radius_s: ti.i32,
    k_o: ti.template(),
    radius_o: ti.i32,
    convolution_storage: ti.template(),
    U_reg: ti.template()
):
    convolve_with_kernel_x_dir(U, k_s, radius_s, convolution_storage)
    convolve_with_kernel_y_dir(convolution_storage, k_s, radius_s, U)
    convolve_with_kernel_θ_dir(U, k_o, radius_o, U_reg)

def regularise_anisotropic(U_np, θs_np, σ1, σ2, σ3, dxy, dθ):
    shape = U_np.shape
    U = ti.field(dtype=ti.f32, shape=shape)
    U.from_numpy(U_np)
    θs = ti.field(dtype=ti.f32, shape=shape)
    θs.from_numpy(θs_np)
    U_reg = ti.field(dtype=ti.f32, shape=shape)
    regularise_anisotropic_ti(U, θs, dxy, dθ, σ1, σ2, σ3, U_reg)
    return U_reg.to_numpy()

@ti.kernel
def regularise_anisotropic_ti(
    U: ti.template(),
    θs: ti.template(),
    dxy: ti.f32,
    dθ: ti.f32,
    σ1: ti.f32,
    σ2: ti.f32,
    σ3: ti.f32,
    U_reg: ti.template()
):
    regularise_anisotropic_int(U, θs, dxy, dθ, σ1, σ2, σ3, U_reg)

In [ ]:
# U = U_pre_R2
# U = U_pre_SE2
U = U_pre_delete

In [ ]:
# σ1, σ2, σ3 = (20. * dxy, 10. * dxy, 2. * dθ)
σ1, σ2, σ3 = (3. * dxy, 1. * dxy, 0.2 * dθ)
truncate = 4
rs = truncate * np.ceil(np.max((σ1, σ2)) / dxy)
ro = truncate * np.ceil(σ3 / dθ)
rs, ro
U_reg = regularise_anisotropic(U, θs, σ1, σ2, σ3, dxy, dθ)
# ν_s = 1.
# ν_o = .5
# U_reg = regularise_isotropic(U, ν_s, ν_o, dxy, dθ)

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"{K} $U$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"{K + 2} $U$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title(f"{K + 4} $U$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"{K} $U$ reg")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_reg[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"{K + 2} $U$ reg")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_reg[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title(f"{K + 4} $U$ reg")
fig.colorbar(cbar, ax=ax[1, 2]);

In [ ]:
test = U.copy()
test[Mask > 0.] = 15.
test_reg = U_reg.copy()
test_reg[Mask > 0.] = 15.

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(test[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"{K} $U$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(test[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"{K + 2} $U$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(test[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title(f"{K + 4} $U$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"{K} $U$ reg")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_reg[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"{K + 2} $U$ reg")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_reg[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title(f"{K + 4} $U$ reg")
fig.colorbar(cbar, ax=ax[1, 2]);

In [ ]:
if save_files:
    filename = f".\\data\\test_switches_SE2_os_aniso_reg.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=U_reg)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(18, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(u, x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title("$u$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_reg.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title("Reconstruction")
fig.colorbar(cbar, ax=ax[1])
_, _, cbar = dsfilter.visualisations.plot_image_array(u - U_reg.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[2])
ax[2].set_title("Reconstruction error")
fig.colorbar(cbar, ax=ax[2]);

In [ ]:
if save_files:
    filename = f".\\data\\test_switches_SE2_os_reg.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=U_reg)

## Check Derivatives

In [ ]:
from dsfilter.SE2.utils import scalar_trilinear_interpolate

In [ ]:
def compute_A1(U_np, θs_np, dxy):
    shape = U_np.shape
    U = ti.field(dtype=ti.f32, shape=shape)
    U.from_numpy(U_np)
    θs = ti.field(dtype=ti.f32, shape=shape)
    θs.from_numpy(θs_np)
    A1_U = ti.field(dtype=ti.f32, shape=shape)
    compute_A1_ti(U, θs, dxy, A1_U)
    return A1_U.to_numpy()

@ti.kernel
def compute_A1_ti(
    U: ti.template(),
    θs: ti.template(),
    dxy: ti.f32,
    A1_U: ti.template()
):
    for I in ti.grouped(U):
        θ = θs[I]
        cos = ti.math.cos(θ)
        sin = ti.math.sin(θ)
        I_A1 = ti.Vector([cos, sin, 0.0], dt=ti.f32) / 2
        A1_U[I] = (scalar_trilinear_interpolate(U, I + I_A1) - scalar_trilinear_interpolate(U, I - I_A1)) / dxy

def compute_A2(U_np, θs_np, dxy):
    shape = U_np.shape
    U = ti.field(dtype=ti.f32, shape=shape)
    U.from_numpy(U_np)
    θs = ti.field(dtype=ti.f32, shape=shape)
    θs.from_numpy(θs_np)
    A2_U = ti.field(dtype=ti.f32, shape=shape)
    compute_A2_ti(U, θs, dxy, A2_U)
    return A2_U.to_numpy()

@ti.kernel
def compute_A2_ti(
    U: ti.template(),
    θs: ti.template(),
    dxy: ti.f32,
    A2_U: ti.template()
):
    for I in ti.grouped(U):
        θ = θs[I]
        cos = ti.math.cos(θ)
        sin = ti.math.sin(θ)
        I_A2 = ti.Vector([-sin, cos, 0.0], dt=ti.f32) / 2
        A2_U[I] = (scalar_trilinear_interpolate(U, I + I_A2) - scalar_trilinear_interpolate(U, I - I_A2)) / dxy

def compute_A3(U_np, dθ):
    shape = U_np.shape
    U = ti.field(dtype=ti.f32, shape=shape)
    U.from_numpy(U_np)
    A3_U = ti.field(dtype=ti.f32, shape=shape)
    compute_A3_ti(U, dθ, A3_U)
    return A3_U.to_numpy()

@ti.kernel
def compute_A3_ti(
    U: ti.template(),
    dθ: ti.f32,
    A3_U: ti.template()
):
    I_A3 = ti.Vector([0.0, 0.0, 1.0], dt=ti.f32) / 2
    for I in ti.grouped(U):
        A3_U[I] = (scalar_trilinear_interpolate(U, I + I_A3) - scalar_trilinear_interpolate(U, I - I_A3)) / dθ

In [ ]:
A1_U = compute_A1(U, θs, dxy)
A2_U = compute_A2(U, θs, dxy)
A3_U = compute_A3(U, dθ)
A1_U_reg = compute_A1(U_reg, θs, dxy)
A2_U_reg = compute_A2(U_reg, θs, dxy)
A3_U_reg = compute_A3(U_reg, dθ)

In [ ]:
K = 2
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(A1_U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("$A_1 U$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(A2_U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("$A_2 U$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(A3_U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title("$A_3 U$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(A1_U_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("$A_1 U_{{reg}}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(A2_U_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title("$A_2 U_{{reg}}$")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(A3_U_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title("$A_3 U_{{reg}}$")
fig.colorbar(cbar, ax=ax[1, 2]);

In [ ]:
if save_files:
    filename = f".\\data\\test_switches_SE2_A1.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=A1_U)
    filename = f".\\data\\test_switches_SE2_A2.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=A2_U)
    filename = f".\\data\\test_switches_SE2_A3.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=A3_U)
    filename = f".\\data\\test_switches_SE2_A1_reg.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=A1_U_reg)
    filename = f".\\data\\test_switches_SE2_A2_reg.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=A2_U_reg)
    filename = f".\\data\\test_switches_SE2_A3_reg.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=A3_U_reg)

## Check Fancy Derivatives

In [ ]:
from dsfilter.SE2.LI.derivatives import (
    gradient_perp,
    laplace_perp
)

In [ ]:
def compute_grad(U_np, θs_np, dxy, dθ, ξ):
    shape = U_np.shape
    U = ti.field(dtype=ti.f32, shape=shape)
    U.from_numpy(U_np)
    θs = ti.field(dtype=ti.f32, shape=shape)
    θs.from_numpy(θs_np)
    gradient_perp_U = ti.field(dtype=ti.f32, shape=shape)
    compute_grad_ti(U, θs, dxy, dθ, ξ, gradient_perp_U)
    return gradient_perp_U.to_numpy()

@ti.kernel
def compute_grad_ti(
    U: ti.template(),
    θs: ti.template(),
    dxy: ti.f32,
    dθ: ti.f32,
    ξ: ti.f32,
    gradient_perp_U: ti.template()
):
    gradient_perp(U, dxy, dθ, θs, ξ, gradient_perp_U)

def compute_laplace(U_np, θs_np, dxy, dθ, ξ):
    shape = U_np.shape
    U = ti.field(dtype=ti.f32, shape=shape)
    U.from_numpy(U_np)
    θs = ti.field(dtype=ti.f32, shape=shape)
    θs.from_numpy(θs_np)
    laplace_perp_U = ti.field(dtype=ti.f32, shape=shape)
    compute_laplace_ti(U, θs, dxy, dθ, ξ, laplace_perp_U)
    return laplace_perp_U.to_numpy()

@ti.kernel
def compute_laplace_ti(
    U: ti.template(),
    θs: ti.template(),
    dxy: ti.f32,
    dθ: ti.f32,
    ξ: ti.f32,
    laplace_perp_U: ti.template()
):
    laplace_perp(U, dxy, dθ, θs, ξ, laplace_perp_U)

In [ ]:
ξ = 8 / dim_K
print(ξ)

In [ ]:
grad_U = compute_grad(U, θs, dxy, dθ, ξ)
laplace_U = compute_laplace(U, θs, dxy, dθ, ξ)
grad_U_reg = compute_grad(U_reg, θs, dxy, dθ, ξ)
laplace_U_reg = compute_laplace(U_reg, θs, dxy, dθ, ξ)

In [ ]:
K = 4
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(grad_U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("grad $U$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(laplace_U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("laplace $U$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(grad_U_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("grad $U_{{reg}}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(laplace_U_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title("laplace $U_{{reg}}$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(grad_U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"{K} grad $U$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(grad_U[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"{K + 2} grad $U$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(grad_U[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title(f"{K + 4} grad $U$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(grad_U_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"{K} grad $U$ reg")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(grad_U_reg[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"{K + 2} grad $U$ reg")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(grad_U_reg[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title(f"{K + 4} grad $U$ reg")
fig.colorbar(cbar, ax=ax[1, 2]);

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(laplace_U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"{K} $\Delta U$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(laplace_U[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"{K + 2} $\Delta U$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(laplace_U[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title(f"{K + 4} $\Delta U$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(laplace_U_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"{K} $\Delta U$ reg")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(laplace_U_reg[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"{K + 2} $\Delta U$ reg")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(laplace_U_reg[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title(f"{K + 4} $\Delta U$ reg")
fig.colorbar(cbar, ax=ax[1, 2]);

In [ ]:
if save_files:
    filename = f".\\data\\test_switches_SE2_grad.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=grad_U)
    filename = f".\\data\\test_switches_SE2_laplace.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=laplace_U)
    filename = f".\\data\\test_switches_SE2_grad_reg.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=grad_U_reg)
    filename = f".\\data\\test_switches_SE2_laplace_reg.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=laplace_U_reg)

## Check Switches

In [ ]:
from dsfilter.SE2.LI.switches import (
    g_scalar,
    S_ε
)

In [ ]:
def compute_DS_switch(gradient_perp_U_np, λ):
    shape = gradient_perp_U_np.shape
    gradient_perp_U = ti.field(dtype=ti.f32, shape=shape)
    gradient_perp_U.from_numpy(gradient_perp_U_np)
    switch = ti.field(dtype=ti.f32, shape=shape)
    compute_DS_switch_ti(gradient_perp_U, λ, switch)
    return switch.to_numpy()

@ti.kernel
def compute_DS_switch_ti(
    gradient_perp_U: ti.template(),
    λ: ti.f32,
    switch: ti.template()
):
    for I in ti.grouped(switch):
        switch[I] = g_scalar(gradient_perp_U[I]**2, λ)

def compute_morphological_switch(laplace_perp_U_np, ε):
    shape = laplace_perp_U_np.shape
    laplace_perp_U = ti.field(dtype=ti.f32, shape=shape)
    laplace_perp_U.from_numpy(laplace_perp_U_np)
    switch = ti.field(dtype=ti.f32, shape=shape)
    compute_morphological_switch_ti(laplace_perp_U, ε, switch)
    return switch.to_numpy()

@ti.kernel
def compute_morphological_switch_ti(
    laplace_perp_U: ti.template(),
    ε: ti.f32,
    switch: ti.template()
):
    for I in ti.grouped(switch):
        switch[I] = (
            S_ε(laplace_perp_U[I], ε) * (ε > 0.) +
            ti.math.sign(laplace_perp_U[I]) * (ε == 0.)
        )

In [ ]:
λ = 1.
ε = 1.

In [ ]:
switch_DS = compute_DS_switch(grad_U, λ)
switch_morph = compute_morphological_switch(laplace_U, ε)
switch_DS_reg = compute_DS_switch(grad_U_reg, λ)
switch_morph_reg = compute_morphological_switch(laplace_U_reg, ε)

In [ ]:
threshold_DS = 0.25
threshold_morph = 0.5
region_of_interest_DS = (switch_DS < threshold_DS) # More shock than diffusion
region_of_interest_morph = (switch_morph < -threshold_morph) + (switch_morph > threshold_morph) # Big shock
region_of_interest_combi = region_of_interest_DS * region_of_interest_morph
region_of_interest_DS_reg = (switch_DS_reg < threshold_DS)
region_of_interest_morph_reg = (switch_morph_reg < -threshold_morph) + (switch_morph_reg > threshold_morph)
region_of_interest_combi_reg = region_of_interest_DS_reg * region_of_interest_morph_reg

In [ ]:
K = 0
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(region_of_interest_DS.astype(int)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("DS")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(region_of_interest_morph.astype(int)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("morph")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(region_of_interest_combi.astype(int)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title("combi")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(region_of_interest_DS_reg.astype(int)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("DS reg")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(region_of_interest_morph_reg.astype(int)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title("morph reg")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(region_of_interest_combi_reg.astype(int)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title("combi reg")
fig.colorbar(cbar, ax=ax[1, 2]);

In [ ]:
K = 4
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("DS $U$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("morph $U$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("DS $U_{{reg}}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title("morph $U_{{reg}}$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"morph {K} $U$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"morph {K + 2} $U$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title(f"morph {K + 4} $U$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"morph {K} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_reg[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"morph {K + 2} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_reg[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title(f"morph {K + 4} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 2]);

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"DS {K} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_reg[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"DS {K + 2} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_reg[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title(f"DS {K + 4} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"morph {K} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_reg[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"morph {K + 2} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_reg[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title(f"morph {K + 4} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 2]);

In [ ]:
if save_files:
    filename = f".\\data\\test_switches_SE2_DS.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=switch_DS)
    filename = f".\\data\\test_switches_SE2_morph.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=switch_morph)
    filename = f".\\data\\test_switches_SE2_DS_reg.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=switch_DS_reg)
    filename = f".\\data\\test_switches_SE2_morph_reg.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=switch_morph_reg)

## Check Switches with External Regularisation

In [ ]:
ρ1 = 4. * dxy
ρ2 = 0.5 * dxy
ρ3 = 0.1 * dθ

In [ ]:
switch_DS_ext = regularise_anisotropic(switch_DS, θs, ρ1, ρ2, ρ3, dxy, dθ)
switch_morph_ext = regularise_anisotropic(switch_morph, θs, ρ1, ρ2, ρ3, dxy, dθ)
switch_DS_reg_ext = regularise_anisotropic(switch_DS_reg, θs, ρ1, ρ2, ρ3, dxy, dθ)
switch_morph_reg_ext = regularise_anisotropic(switch_morph_reg, θs, ρ1, ρ2, ρ3, dxy, dθ)

In [ ]:
threshold_DS = 0.5
threshold_morph = 0.25
region_of_interest_DS_ext = (switch_DS_ext < threshold_DS) # More shock than diffusion
region_of_interest_morph_ext = (switch_morph_ext < -threshold_morph) + (switch_morph_ext > threshold_morph) # Big shock
region_of_interest_combi_ext = region_of_interest_DS_ext * region_of_interest_morph_ext
region_of_interest_DS_reg_ext = (switch_DS_reg_ext < threshold_DS)
region_of_interest_morph_reg_ext = (switch_morph_reg_ext < -threshold_morph) + (switch_morph_reg_ext > threshold_morph)
region_of_interest_combi_reg_ext = region_of_interest_DS_reg_ext * region_of_interest_morph_reg_ext

In [ ]:
K = 0
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(region_of_interest_DS_ext.astype(int)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title("DS")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(region_of_interest_morph_ext.astype(int)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("morph")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(region_of_interest_combi_ext.astype(int)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title("combi")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(region_of_interest_DS_reg_ext.astype(int)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title("DS reg")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(region_of_interest_morph_reg_ext.astype(int)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title("morph reg")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(region_of_interest_combi_reg_ext.astype(int)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title("combi reg")
fig.colorbar(cbar, ax=ax[1, 2]);

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_ext[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"DS {K} $U$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_ext[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"DS {K + 2} $U$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_ext[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title(f"DS {K + 4} $U$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_reg_ext[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"DS {K} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_reg_ext[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"DS {K + 2} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_reg_ext[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title(f"DS {K + 4} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 2]);

In [ ]:
test = switch_DS_ext.copy()
test[Mask > 0.] = 1.
test_reg = switch_DS_reg_ext.copy()
test_reg[Mask > 0.] = 1.

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(test[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"DS {K} $U$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(test[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"DS {K + 2} $U$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(test[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title(f"DS {K + 4} $U$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"DS {K} $U$ reg")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_reg[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"DS {K + 2} $U$ reg")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_reg[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title(f"DS {K + 4} $U$ reg")
fig.colorbar(cbar, ax=ax[1, 2]);

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_ext[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"morph {K} $U$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_ext[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"morph {K + 2} $U$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_ext[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title(f"morph {K + 4} $U$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_reg_ext[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"morph {K} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_reg_ext[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"morph {K + 2} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_reg_ext[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title(f"morph {K + 4} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 2]);

In [ ]:
test = switch_morph_ext.copy()
test[Mask > 0.] = 0.
test_reg = switch_morph_reg_ext.copy()
test_reg[Mask > 0.] = 0.

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(test[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"morph {K} $U$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(test[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"morph {K + 2} $U$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(test[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title(f"morph {K + 4} $U$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_reg[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"morph {K} $U$ reg")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_reg[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"morph {K + 2} $U$ reg")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(test_reg[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title(f"morph {K + 4} $U$ reg")
fig.colorbar(cbar, ax=ax[1, 2]);

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 3, figsize=(18, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_reg_ext[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"DS {K} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_reg_ext[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title(f"DS {K + 2} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_DS_reg_ext[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 2])
ax[0, 2].set_title(f"DS {K + 4} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[0, 2])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_reg_ext[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"morph {K} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_reg_ext[..., K + 2], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title(f"morph {K + 2} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(switch_morph_reg_ext[..., K + 4], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 2])
ax[1, 2].set_title(f"morph {K + 4} $U_{{\mathrm{{reg}}}}$")
fig.colorbar(cbar, ax=ax[1, 2]);

In [ ]:
if save_files:
    filename = f".\\data\\test_switches_SE2_DS_ext.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=switch_DS_ext)
    filename = f".\\data\\test_switches_SE2_morph_ext.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=switch_morph_ext)
    filename = f".\\data\\test_switches_SE2_DS_reg_ext.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=switch_DS_reg_ext)
    filename = f".\\data\\test_switches_SE2_morph_reg_ext.hdf5"
    with h5py.File(filename, "w") as distance_file:
        distance_file.create_dataset("OS", data=switch_morph_reg_ext)